## A Model to Predict Premier League Games using Poisson Distribution.

In [1]:
import pandas as pd 
import math 
import matplotlib.pyplot as plt
from ipywidgets import widgets as wg
from IPython.display import display, clear_output 

In [2]:
pd_tables = pd.read_html("https://thefishy.co.uk/leaguetable.php?table=1")
league_table = pd_tables[1];

In [3]:
teams_list = league_table["Team"][0:20].values.tolist()
league_ave_home_goal = league_table["HF"].mean() / league_table["P"].mean()
league_ave_home_conc = league_table["HA"].mean() / league_table["P"].mean()

league_ave_away_goal = league_table["AF"].mean() / league_table["P"].mean()
league_ave_away_conc = league_table["AA"].mean() / league_table["P"].mean()

In [4]:
strength_list = {}

def calc_strength(home_team, away_team):

    home_team_index = teams_list.index(home_team) 
    away_team_index = teams_list.index(away_team) 

    home_team_game_played = league_table["P"][home_team_index]
    away_team_game_played = league_table["P"][away_team_index]

    home_team_goal = league_table["HF"][home_team_index]
    home_team_conc = league_table["HA"][home_team_index]
    away_team_goal = league_table["AF"][away_team_index]
    away_team_conc = league_table["AA"][away_team_index]

    # home team attack strength at home              
    home_team_attack_strength = (home_team_goal / home_team_game_played) / league_ave_home_goal

    # home team defense strength at home
    home_team_defense_strength = (home_team_conc / home_team_game_played) / league_ave_home_conc

    # away team attack strength at away
    away_team_attack_strength = (away_team_goal / away_team_game_played) / league_ave_away_goal

    # away team defense strength at away
    away_team_defense_strength = (away_team_conc / away_team_game_played) / league_ave_away_conc

    global strength_list
    strength_list["home_attack_str"] = home_team_attack_strength
    strength_list["home_defense_str"] = home_team_defense_strength
    strength_list["away_attack_str"] = away_team_attack_strength
    strength_list["away_defense_str"] = away_team_defense_strength

In [5]:
exp_goal_ls = {}

def calc_goal_expectancy(strength_list):
    
    home_goal_exp = strength_list["home_attack_str"] * strength_list["away_defense_str"] * league_ave_home_goal
    away_goal_exp = strength_list["away_attack_str"] * strength_list["home_defense_str"] * league_ave_away_goal

    global exp_goal_ls
    exp_goal_ls["home_exp_goal"] = home_goal_exp
    exp_goal_ls["away_exp_goal"] = away_goal_exp

In [6]:
home_goal_prob = {}
away_goal_prob = {}

def scoring_probability(exp_goal_ls):

    for i in range(10):
        
        home_goal_prob[i] = float(((exp_goal_ls["home_exp_goal"]**i * (math.e**( -exp_goal_ls["home_exp_goal"]))) 
        / math.factorial(i))) 
        
        away_goal_prob[i] = float(((exp_goal_ls["away_exp_goal"]**i * (math.e**( -exp_goal_ls["away_exp_goal"]))) 
        / math.factorial(i)))

In [7]:
score_chances = {}

def result_chances(home_goal_exp, away_goal_exp):

    home_win_perc = 0;
    draw_perc = 0;
    away_win_perc = 0;
    
    for i in range(10):
        for j in range(10): 
            score_chances[str(i)+'-'+str(j)] = ((home_goal_exp[i] * away_goal_exp[j]) * 100) 
            if i > j:
                home_win_perc += score_chances[str(i)+'-'+str(j)]
            elif i < j:
                away_win_perc += score_chances[str(i)+'-'+str(j)]
            else:
                draw_perc += score_chances[str(i)+'-'+str(j)]

    result_chances_ls = {"home_win": home_win_perc, "draw_perc":draw_perc, "away_win": away_win_perc, }  

    return result_chances_ls

In [8]:
def plot_result_chances(home_team_name, away_team_name, result_chances_ls):
    fig = plt.figure(figsize=(10, 6))
    results_text = [home_team_name + " Win", "Draw", away_team_name + " Win"]
    results_perc = [result_chances_ls["home_win"], result_chances_ls["draw_perc"], result_chances_ls["away_win"]]
    graph = plt.bar(results_text, results_perc, color = ['cornflowerblue','wheat','lightcoral'])
    for bar in graph:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, yval, int(yval))
            
    plt.xlabel("Results", fontsize = 14)
    plt.ylabel("Percentage of Chances (%)", fontsize = 14)
    plt.title("Percentage of Chances of Win or Draw", fontsize = 16)
    plt.show()

In [9]:
done_btn = wg.Button(description = 'Done')
home_team = wg.Dropdown(description = "Home",options = teams_list, value = teams_list[0])
away_team = wg.Dropdown(description = "Away",options = teams_list, value = teams_list[1])
out = wg.Output()

display(home_team,away_team,done_btn,out)

def done(arg):
    with out:
        clear_output()
        if home_team.value == away_team.value:
            print("Same Team")
        else:
            calc_strength(home_team.value, away_team.value)
            calc_goal_expectancy(strength_list)
            scoring_probability(exp_goal_ls)
            result = result_chances(home_goal_prob, away_goal_prob)
            plot_result_chances(home_team.value, away_team.value, result)
            
done_btn.on_click(done) 

Dropdown(description='Home', options=('Arsenal', 'Man City', 'Man Utd', 'Newcastle', 'Tottenham', 'Brighton', …

Dropdown(description='Away', index=1, options=('Arsenal', 'Man City', 'Man Utd', 'Newcastle', 'Tottenham', 'Br…

Button(description='Done', style=ButtonStyle())

Output()